In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True, index_col='TransactionID')
    df = reduce_mem_usage(df)
    return df

In [ ]:
%%time

print('Loading data...')

train_id = import_data("../input/ieee-fraud-detection/train_identity.csv")
print('\tSuccessfully loaded train_identity!')

X_train = import_data('../input/ieee-fraud-detection/train_transaction.csv')
print('\tSuccessfully loaded train_transaction!')
X_train = X_train.merge(train_id, how='left', left_index=True, right_index=True) # Train setini kendi içinde merge etmiş


test_id = import_data('../input/ieee-fraud-detection/test_identity.csv')
print('\tSuccessfully loaded test_identity!')

X_test = import_data('../input/ieee-fraud-detection/test_transaction.csv')
print('\tSuccessfully loaded test_transaction!')

test_id.columns = train_id.columns
X_test = X_test.merge(test_id, how='left', left_index=True, right_index=True)  # Test setini kendi içinde merge etmiş

pd.set_option('max_columns', None)

# TARGET
y_train = X_train['isFraud'].copy()  # Train deki bağımlı değişkeni y_train setine atamış.


print('Data was successfully loaded!\n')

**PREPROCESSING**

In [ ]:
# CARD 1-6 

valid_card = pd.concat([X_train[['card1']], X_test[['card1']]])
valid_card = valid_card['card1'].value_counts()
valid_card_std = valid_card.values.std()

invalid_cards = valid_card[valid_card<=2]

valid_card = valid_card[valid_card>2]
valid_card = list(valid_card.index)

X_train['card1'] = np.where(X_train['card1'].isin(X_test['card1']), X_train['card1'], np.nan)
X_test['card1']  = np.where(X_test['card1'].isin(X_train['card1']), X_test['card1'], np.nan)

X_train['card1'] = np.where(X_train['card1'].isin(valid_card), X_train['card1'], np.nan)
X_test['card1']   = np.where(X_test['card1'].isin(valid_card), X_test['card1'], np.nan)

# burda frekans sayısı 2 den az olan kartlara invalid çok olanlara valid kart demiş
# sonra train ve testin ikisinde de bulunanları almış eğer birinde bulunmuyorsa nan yapmış
# sonra invalid olanları nan yapmış


for col in ['card2','card3','card4','card5','card6']: 
    X_train[col] = np.where(X_train[col].isin(X_test[col]), X_train[col], np.nan)
    X_test[col]  = np.where(X_test[col].isin(X_train[col]), X_test[col], np.nan)

# train ve testin ikisinde de bulunanları almış eğer birinde bulunmuyorsa nan yapmış

In [ ]:
# id_30 - ID_31 

X_train['OS_id_30'] = X_train['id_30'].str.split(' ', expand=True)[0]
X_train['version_id_30'] = X_train['id_30'].str.split(' ', expand=True)[1]

X_test['OS_id_30'] = X_test['id_30'].str.split(' ', expand=True)[0]
X_test['version_id_30'] = X_test['id_30'].str.split(' ', expand=True)[1]


X_train['browser_id_31'] = X_train['id_31'].str.split(' ', expand=True)[0]
X_train['version_id_31'] = X_train['id_31'].str.split(' ', expand=True)[1]

X_train['browser_id_31'] = X_train['id_31'].str.split(' ', expand=True)[0]
X_test['version_id_31'] = X_test['id_31'].str.split(' ', expand=True)[1]

# cihaz ve browser tespitinin onemli oldugu varsayimiyla yapildi...

In [ ]:
# TransactionAmt

X_train['TransactionAmt'] = X_train['TransactionAmt'].astype('float32')

X_train['Trans_min_std'] = (X_train['TransactionAmt'] - X_train['TransactionAmt'].mean()) / X_train['TransactionAmt'].std()

X_test['Trans_min_std'] = (X_test['TransactionAmt'] - X_test['TransactionAmt'].mean()) / X_test['TransactionAmt'].std()

# amt ilk halinde float16, bu sekilde std ve mean NAN oluyor, float32 yapmamiz lazim

**FEATURE ENGINEERING**

In [ ]:
# lastest_browser (SON VERSIYON KONTROLU)

X_train["lastest_browser"] = np.zeros(X_train.shape[0])
X_test["lastest_browser"] = np.zeros(X_test.shape[0])

def setBrowser(df):
    df.loc[df["id_31"]=="samsung browser 7.0",'lastest_browser']=1
    df.loc[df["id_31"]=="opera 53.0",'lastest_browser']=1
    df.loc[df["id_31"]=="mobile safari 10.0",'lastest_browser']=1
    df.loc[df["id_31"]=="google search application 49.0",'lastest_browser']=1
    df.loc[df["id_31"]=="firefox 60.0",'lastest_browser']=1
    df.loc[df["id_31"]=="edge 17.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 69.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 67.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 63.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 63.0 for ios",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 64.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 64.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 64.0 for ios",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 65.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 65.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 65.0 for ios",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 66.0",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 66.0 for android",'lastest_browser']=1
    df.loc[df["id_31"]=="chrome 66.0 for ios",'lastest_browser']=1
    return df

X_train=setBrowser(X_train)
X_test=setBrowser(X_test)

# son versiyon kontrolu, son versiyon olanlar 1 seklinde belirtilmis

In [ ]:
# CARD1 - CARD5 KULLANIM MIKTARLARI

for feature in ['card1','card5']:
    X_train[feature + '_count_full'] = X_train[feature].map(pd.concat([X_train[feature], X_test[feature]], ignore_index=True).value_counts(dropna=False))
    X_test[feature + '_count_full'] = X_test[feature].map(pd.concat([X_train[feature], X_test[feature]], ignore_index=True).value_counts(dropna=False))
    
# card larin kullanim yogunlugunun onemli olabilecegini varsayarak bu islem yapildi, sahtekarlik yapanlar yogun kullanilan kartlari... card 1 ve 5 in ozgun oldugu goruldu

In [ ]:
# MAIL ADRESLERININ SON KISIMLARINDAN ULKE TESPITINE YONELIK URETILEN FEATURE (com, us, mx, es, de, fr, uk, jp)

us_emails = ['gmail', 'net', 'edu']

for c in ['P_emaildomain', 'R_emaildomain']:
    
    X_train[c + '_suffix'] = X_train[c].map(lambda x: str(x).split('.')[-1])
    X_test[c + '_suffix'] = X_test[c].map(lambda x: str(x).split('.')[-1])
    
    X_train[c + '_suffix'] = X_train[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    X_test[c + '_suffix'] = X_test[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

In [ ]:
p = 'P_emaildomain'
r = 'R_emaildomain'
unknown = 'email_not_provided'

def setDomain(df):
    df[p] = df[p].astype('str')
    df[r] = df[r].astype('str')
    
    df[p] = df[p].fillna(unknown)
    df[r] = df[r].fillna(unknown)
    
    df['email_check'] = np.where((df[p]==df[r])&(df[p]!=unknown),1,0)

    df[p+'_prefix'] = df[p].apply(lambda x: x.split('.')[0])
    df[r+'_prefix'] = df[r].apply(lambda x: x.split('.')[0])
    
    return df
    
X_train=setDomain(X_train)
X_test=setDomain(X_test)

# Check if P_emaildomain matches R_emaildomain 
# extracts prefix

In [ ]:
# TransactionDT degerlerinden icin yeni degiskenler uretilmis.

import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# start='2017-10-01', end='2019-01-01 arasindaki tarihler listelenmis.
dates_range = pd.date_range(start='2017-10-01', end='2019-01-01')

# start='2017-10-01', end='2019-01-01 ABD ulusal tatil gunleri listelenmis. 
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")


# islemlerin yapildigi hour of day, day of week ve day of month ve month of year degiskeni olusturulmus.

X_train["Date"] = X_train['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
X_train['_Weekdays'] = X_train['Date'].dt.dayofweek
X_train['_Dayhours'] = X_train['Date'].dt.hour
X_train['_Monthdays'] = X_train['Date'].dt.day
X_train['_Yearmonths'] = (X_train['Date'].dt.month).astype(np.int8) 

X_test["Date"] = X_test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
X_test['_Weekdays'] = X_test['Date'].dt.dayofweek
X_test['_Dayhours'] = X_test['Date'].dt.hour
X_test['_Monthdays'] = X_test['Date'].dt.day
X_test['_Yearmonths'] = (X_test['Date'].dt.month).astype(np.int8) 


# yapilan islem tatil gunumu mu?

X_train['is_holiday'] = (X_train['Date'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
X_test['is_holiday'] = (X_test['Date'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)

In [ ]:
# Timestamp tipinde olduğu için algoritma tanımlayamıyor.
X_train.drop("Date", axis=1,inplace=True)
X_test.drop("Date", axis=1,inplace=True)

#X_train.drop("DT_1", axis=1,inplace=True)
#X_test.drop("DT_1", axis=1,inplace=True)

In [ ]:
# ProductCD value_count = (W,C,R,H,S) 
# M4 value_count = (M0,M1,M2)

# kategorik degisken olan ProductCD ve M4, 'fraud' ortalamalarina gore gruplandiriliyor

for col in ['ProductCD','M4']:
    temp_dict = X_train.groupby([col])['isFraud'].agg(['mean']).reset_index().rename(columns={'mean': col+'_target_mean'})
    temp_dict.index = temp_dict[col].values
    temp_dict = temp_dict[col+'_target_mean'].to_dict()
    
    if col=='ProductCD':
        X_train['ProductCD_1'] = X_train[col].map(temp_dict)
        X_test['ProductCD_1']  = X_test[col].map(temp_dict)
    else:
        X_train['M4_1'] = X_train[col].map(temp_dict)
        X_test['M4_1']  = X_test[col].map(temp_dict)

In [ ]:
X_train['TransactionAmt_decimal_lenght'] = X_train['TransactionAmt'].astype(str).str.split('.', expand=True)[1].str.len()
X_test['TransactionAmt_decimal_lenght'] = X_test['TransactionAmt'].astype(str).str.split('.', expand=True)[1].str.len()

#dolar kuruna gore ulke tahmini

In [ ]:
X_train.drop("isFraud", axis=1, inplace=True)

In [ ]:
# Modelde D ve D' nin normalize edilmiş  kolonlarının çıkarılmış halini de deneyeceğiz.
# The D Columns are "time deltas" from some point in the past. We will transform the D Columns into their point in the past.
# NORMALIZE D COLUMNS
for i in range(1,16):
    if i in [1,2,3,5,9]:
        continue
    X_train['D'+str(i)] =  X_train['D'+str(i)] - X_train.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(i)] = X_test['D'+str(i)] - X_test.TransactionDT/np.float32(24*60*60)

In [ ]:
# BURASI KENDİMİZCE YAZILACAK VE FONKSİYON OLARAK TANIMLANACAK.

"""

#### FEATURE ENGINEERING ISLEMLERINDEN USERID BELIRLENDIKTEN SONRA USERID'YE GORE YAPILACAK AGGREGATION ISLEMLERININ SON HALINE BURADA KARAR VERILECEK

X_train['mean_last'] = X_train['TransactionAmt'] - X_train.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).mean())
X_train['min_last'] = X_train.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).min())
X_train['max_last'] = X_train.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).max())
X_train['std_last'] = X_train['mean_last'] / X_train.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).std())

X_train['mean_last'].fillna(0, inplace=True, )
X_train['std_last'].fillna(0, inplace=True)



X_test['mean_last'] = X_test['TransactionAmt'] - X_test.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).mean())
X_test['min_last'] = X_test.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).min())
X_test['max_last'] = X_test.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).max())
X_test['std_last'] = X_test['mean_last'] / X_test.groupby('userid')['TransactionAmt'].transform(lambda x: x.rolling(10, 1).std())

X_test['mean_last'].fillna(0, inplace=True, )
X_test['std_last'].fillna(0, inplace=True)




X_train['TransactionAmt_to_mean_card_id'] = X_train['TransactionAmt'] - X_train.groupby(['userid'])['TransactionAmt'].transform('mean')
X_train['TransactionAmt_to_std_card_id'] = X_train['TransactionAmt_to_mean_card_id'] / X_train.groupby(['userid'])['TransactionAmt'].transform('std')
X_test['TransactionAmt_to_mean_card_id'] = X_test['TransactionAmt'] - X_test.groupby(['userid'])['TransactionAmt'].transform('mean')
X_test['TransactionAmt_to_std_card_id'] = X_test['TransactionAmt_to_mean_card_id'] / X_test.groupby(['userid'])['TransactionAmt'].transform('std')



X_train = X_train.replace(np.inf,999)# sonsuz değerleri 999 ile değiştiriyor
X_test = X_test.replace(np.inf,999)

"""


# GROUP AGGREGATION MEAN AND STD
# https://www.kaggle.com/kyakovlev/ieee-fe-with-some-eda
# AMAÇ: seçilen değişkenlerin uids lere göre gruplanıp ortalamaların alıp yeni değişkene atamak.
def encode_AG(main_columns, uids, aggregations=['mean'], train_df=X_train, test_df=X_test, 
              fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type # sectigi kolon isimlerini ve aggregation degerini birleştirip yeni değişken ismi oluşturmuş
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]]) # Train ve Test setten seçtiği değişkenleri alt alta birleştirmiş.
                if usena: 
                    temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan # main_colums dan gelen değişkende -1 olan değerleri nan yapar.
                # col değişkenine göre groupby atmış ve main_column değişkenine agg_type türüne göre işlem yapmış ve bunu new_col_name isimli yeni değişkene atamış.İNDEX İ DE SIFIRLADI.
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})
                temp_df.index = list(temp_df[col]) # temp_df nin sıfırlanan indexleri col değişkeninin index leriyle değiştirildi.
                temp_df = temp_df[new_col_name].to_dict()  # Yni oluşturulan değişken sözlük türüne çevirildi.
                 
                # Bu yeni değerler Train ve Test setlerinde ilgili kolona(col değişkeni) karşılık gelecek şekilde  "new_col_name" ismiyle Train ve Test sete eklendi.
                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
                # Yeni olusturulan degiskenlerdeki nan değerler yerine -1 yazdırır.
                if fillna:
                    train_df[new_col_name].fillna(-1,inplace=True)
                    test_df[new_col_name].fillna(-1,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')

      
    
    
 # GROUP AGGREGATION NUNIQUE           
            
# AMAÇ: uids lere göre main_colums daki değişkenler gruplanır ve ve bu main_column daki farklı değerler sayılır ve 
# bu toplam sayı Test ve Trainde ilgili "TransactionID" nin karşısına atanır.

def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:  
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            #Burada mp aynı col değişkenine denk gelen main_column değişkenindeki farklı değerlerin toplam sayısıdır.
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            
            # "uid_P_emaildomain_ct" şeklinde değişken oluşturulur ve Train ile test setinde col değişkenindeki değerlere "TransactionID" baz alınarak mp değerleri eşlenir.
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            
            print(col+'_'+main_column+'_ct, ',end='')


### NOT: BURADAKİ LABEL ENCODERLARDAN 1 İNİ SEÇ VE ALTTAKİ ELİF KISMINI AYRI BİR FOR DÖNGÜSÜ NDE NUMERİKLERE İŞLEM YAPACAK ŞEKİLDE TANIMLA

In [ ]:
%%time

# SHIFT ALL NUMERICS POSITIVE. SET NAN to -1 
def num_positiv(X_train,X_test):
    for f in X_train.columns:  
        # Bütün nümerik değerleri pozitif yap ve NAN değerleri -1 yap. ['TransactionAmt','TransactionDT'] kolonları hariç.
        if f not in ['TransactionAmt','TransactionDT',"isFraud"]: 
            mn = np.min((X_train[f].min(),X_test[f].min()))  # X_train ve X_test deki f kolonunun minimum degerlerini kıyasla ve en küçük olanını "mn" ye ata.

            # Buradaki amaç bütün değerlerden en küçük değeri çıkartarak onları pozitif yaparken aralarındaki değer farkınıda korumaktır.
            X_train[f] -= np.float32(mn)   # X_train deki f kolonundaki değerlerden mn yi yani en küçük değeri çıkarır.
            X_test[f] -= np.float32(mn)    # X_test deki f kolonundaki değerlerden mn yi yani en küçük değeri çıkarır.
            X_train[f].fillna(-1,inplace=True)  # X_train deki NaN değerleri -1 ile doldurur.
            X_test[f].fillna(-1,inplace=True)   # X_test deki NaN değerleri -1 ile doldurur.

In [ ]:

# FREQUENCY ENCODE TOGETHER
# AMAÇ: encode_FE fonksiyonu girilen data setlerindeki belirtilen kolonları normalize edip türlerini "float32" ye çevirip _FE uzantılı yeni bir değişken olarak data setlerine ekler.
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True).to_dict() # col. kolonundaki unique değerleri alıp bunları normalize ediyor ve listeye çevirip vc değişkeninde saklıyor.
        vc[-1] = -1       # vc.  sözlüğüne -1 key adı ile -1 değerini ekliyor.
        nm = col+'_FE'    # kolon isimlerine uyguladığı FE encode uzantısını ekliyor.
        df1[nm] = df1[col].map(vc) #vc deki keys değerleri ile df1[col] daki index değerlerini eşleyip karşılığına vc deki values değerlerini atayıp bunu df1'e yeni değişken olarak atar.
        df1[nm] = df1[nm].astype('float32') # yeni değişkenin türünü "float32" yapar.
        df2[nm] = df2[col].map(vc) #vc deki keys değerleri ile df2[col] daki index değerlerini eşleyip karşılığına vc deki values değerlerini atayıp bunu df2'ye yeni değişken olarak atar.
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')
        

# LABEL ENCODE
# AMAÇ: BURADA her bir kolondaki unique değişkene bir factorize ile bir kod atamış ve bunu "df_comb" de saklamış. 
# df_comb de her bir uniqe değişkene 0 dan başlayıp 1 artırarak değer veriyor. Çoklayan değerlere aynı unique değeri atıyor.
# "_" de ise col dan gelen gerçek değerleri saklıyor. 
# Kolonlardaki max değeri 32000 den küçükse bu kolonun türünü "int16" yapmış aksi halde "int32" yapmış.
def encode_LE(col,train=X_train,test=X_test,verbose=True, val=2):
    
    if val ==2:
        df_comb = pd.concat([train[col],test[col]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        nm = col
        if df_comb.max()>32000: 
            train[nm] = df_comb[:len(train)].astype('int32')
            test[nm] = df_comb[len(train):].astype('int32')
        else:
            train[nm] = df_comb[:len(train)].astype('int16')
            test[nm] = df_comb[len(train):].astype('int16')
        del df_comb; x=gc.collect()
        if verbose: 
            print(nm,', ',end='')
            
    else:
        # LABEL ENCODE AND MEMORY REDUCE
        for f in  col:  
    # FACTORIZE CATEGORICAL VARIABLES
            if (np.str(X_train[f].dtype)=='category')|(X_train[f].dtype=='object'): # seçilen f kolonu object veya kategorik ise
                df_comb = pd.concat([X_train[f],X_test[f]],axis=0)     # X_train ve X_test in f kolon değerlerini alt alta birleştir.
                df_comb,_ = df_comb.factorize(sort=True)  # Burada factorize ile f kolonundaki her bir unique değere bir unique ıd atıyor. df_comb ise kolondaki değerleri alır.(label encode)
                if df_comb.max()>32000:  # Eğer f kolonunun max değeri 32000 den büyükse f in int32 ye cevrilmesi lazım yazdır. 
                    train[f] = df_comb[:len(train)].astype('int32')
                    test[f] = df_comb[len(train):].astype('int32')
                else:        
                    X_train[f] = df_comb[:len(X_train)].astype('int16')   # X_train kategorik kolonunun dtype ını int16 ya cevir.
                    X_test[f] = df_comb[len(X_train):].astype('int16')    # X_test kategorik kolonunun dtype ını int16 ya cevir.

        

                
                
                
# COMBINE FEATURES
def encode_CB(col1,col2,df1=X_train,df2=X_test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)   # 12926.0_215.0, 3663.0_230.0  şeklinde çıktı üretir.
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    
    encode_LE(nm,verbose=False) # yukarıdaki değerleri 44945, 60885 şeklinde gibi pozitif çıktıya dönüştürür.Ayrıntılı bilgi için encode_LE ye bakın.
    print(nm,', ',end='')
    
    
    


        
        



# YUKARIDAKİ FONKSİYONLARIN KULLANILDIĞI ALAN

In [ ]:
X_train.drop("browser_id_31", axis = 1, inplace = True)

In [ ]:
%%time
# Burada 'TransactionAmt' değerlerinin küsüratlı kısımlarını alıp "cents" adlı değişkene atadı.
# TRANSACTION AMT CENTS
X_train['cents'] = (X_train['TransactionAmt'] - np.floor(X_train['TransactionAmt'])).astype('float32')
X_test['cents'] = (X_test['TransactionAmt'] - np.floor(X_test['TransactionAmt'])).astype('float32')
print('cents, ', end='')


# FREQUENCY ENCODE: ADDR1, CARD1, CARD2, CARD3, P_EMAILDOMAIN
# AMAÇ: encode_FE fonksiyonu girilen data setlerindeki belirtilen kolonları normalize edip türlerini "float32" ye çevirip _FE uzantılı yeni bir değişken olarak data setlerine ekler.
encode_FE(X_train,X_test,['addr1','card1','card2','card3','P_emaildomain'])

# COMBINE COLUMNS CARD1+ADDR1, CARD1+ADDR1+P_EMAILDOMAIN
encode_CB('card1','addr1')
encode_CB('card1_addr1','P_emaildomain')

# FREQUENCY ENOCDE
encode_FE(X_train,X_test,['card1_addr1','card1_addr1_P_emaildomain'])

# GROUP AGGREGATE
encode_AG(['TransactionAmt','D9','D11'],['card1','card1_addr1','card1_addr1_P_emaildomain'],['mean','std'],usena=True)

print(1)

import datetime
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
X_train['DT_M'] = X_train['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
X_train['DT_M'] = (X_train['DT_M'].dt.year-2017)*12 + X_train['DT_M'].dt.month 

X_test['DT_M'] = X_test['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
X_test['DT_M'] = (X_test['DT_M'].dt.year-2017)*12 + X_test['DT_M'].dt.month 

print(2)

# X_train['uid'] ler 10230_-13.0, 24904_-111.0 şeklinde değerler alacak.
X_train['day'] = X_train.TransactionDT / (24*60*60)
X_train['uid'] = X_train.card1_addr1.astype(str)+'_'+np.floor(X_train.day-X_train.D1).astype(str)


X_test['day'] = X_test.TransactionDT / (24*60*60)
X_test['uid'] = X_test.card1_addr1.astype(str)+'_'+np.floor(X_test.day-X_test.D1).astype(str)

print(3)
# LABEL ENCODE AND MEMORY REDUCE
encode_LE(X_train.columns,train=X_train,test=X_test,verbose=True, val=1)
print(4)
# SHIFT ALL NUMERICS POSITIVE. SET NAN to -1 
num_positiv(X_train,X_test)
print(5)

# FREQUENCY ENCODE UID
encode_FE(X_train,X_test,['uid'])
print(6)
# AGGREGATE 
encode_AG(['TransactionAmt','D4','D9','D10','D15'],['uid'],['mean','std'],fillna=True,usena=True)
print(7)

# AGGREGATE
# Burada C değişkenlerini (c3 hariç) uid e göre gruplayıp mean değerlerini almış.
encode_AG(['C'+str(x) for x in range(1,15) if x!=3],['uid'],['mean'],X_train,X_test,fillna=True,usena=True)
print(8)
# AGGREGATE
# Burada M değişkenlerini uid e göre gruplayıp mean değerlerini almış.
encode_AG(['M'+str(x) for x in range(1,10)],['uid'],['mean'],fillna=True,usena=True)
print(9)
# AGGREGATE
# Bu değişkenleri uid e göre gruplamış ve içlerindeki her bir unique değişkene bir sıfırdan başlayarak bir değer vermiş. 
# sonra bu değerleri train ve test setteki yerlerine yayeni değişken olarak atamış.
encode_AG2(['P_emaildomain','dist1','DT_M','id_02','cents'], ['uid'], train_df=X_train, test_df=X_test)
print(10)
# AGGREGATE
encode_AG(['C14'],['uid'],['std'],X_train,X_test,fillna=True,usena=True)
print(11)
# AGGREGATE 
encode_AG2(['C13','V314'], ['uid'], train_df=X_train, test_df=X_test)
print(12)
# AGGREATE 
encode_AG2(['V127','V136','V309','V307','V320'], ['uid'], train_df=X_train, test_df=X_test)
print(13)
# NEW FEATURE
# Burada D1 den D15 i çıkartarak mutlak değerini almış ve değeri 3 ten büyük olanları int8 formatına çevirip train ve test setlerine yeni değişken olarak atamış.
X_train['outsider15'] = (np.abs(X_train.D1-X_train.D15)>3).astype('int8')
X_test['outsider15'] = (np.abs(X_test.D1-X_test.D15)>3).astype('int8')
print('outsider15')

In [ ]:
"""
####### Clip max values
for df in [X_train, X_test]:
    for col in i_cols:
        max_value = df[df['DT_M']==df['DT_M'].max()][col].max()
        df[col] = df[col].clip(None,max_value)

"""


In [ ]:
X_train.head()

In [ ]:
# Burada ilk modelde yapmış olduğu testler sonucu tespit etmiş olduğu featureları çıkartmış.

cols = list( X_train.columns )
cols.remove('TransactionDT')
for c in ['D6','D7','D8','D9','D12','D13','D14']:
    if c in cols:
        cols.remove(c)
    
# Bu değişkenleri gruplamalar için kullandıktan sonra çıkartıyor. 
for c in ["DT_M",'day','uid','Card_ID','uid1','uid2','uid3','uid4','uid5']:
    if c in cols:
        cols.remove(c)
    
# FAILED TIME CONSISTENCY TEST (19 değişkeni çıkartır)
for c in ['C3','M5','id_08','id_33']:
    if c in cols:
        cols.remove(c)
for c in ['card4','id_07','id_14','id_21','id_30','id_32','id_34']:
    if c in cols:
        cols.remove(c)
for c in ['id_'+str(x) for x in range(22,28)]:
    if c in cols:
        cols.remove(c)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# NAN DEĞERLER VARSA BU DEĞERLERİN MODELDEN ÖNCE BİR ŞEYLE DEĞİŞTİRİLMESİ LAZIM. YUKARIDA -999 İLE DEĞİŞTİRMİŞTİK.
X_train = X_train.fillna(-1)
X_test = X_test.fillna(-1)

In [ ]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

# MODEL KISMI

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

# Hyperparameters Tuning and Best fFeature Selection

In [ ]:
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

def auc(m, train, test,y_train1,y_test1): 
    return (metrics.roc_auc_score(y_train1,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test1,m.predict_proba(test)[:,1]))

# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [8,12],
              "min_child_weight" : [1],
              "n_estimators": [200],
              "learning_rate": [0.14,0.2, 0.25],
              "nthread":[4],
              "tree_method":["gpu_hist"],
              "random_state": [2]}
                             
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10,n_jobs =-1)

In [ ]:
grid_search.fit(X_train1, y_train1)

In [ ]:
grid_search.best_estimator_

In [ ]:
max_depth_best = grid_search.best_estimator_.max_depth
n_estimators_best = grid_search.best_estimator_.n_estimators
learning_rate_best = grid_search.best_estimator_.learning_rate
min_child_weight_best = grid_search.best_estimator_.min_child_weight

In [ ]:
import xgboost as xgb
print("XGBoost version:", xgb.__version__)

if BUILD96:
    clf = xgb.XGBClassifier( 
        n_estimators=n_estimators_best,
        max_depth=max_depth_best, 
        learning_rate=learning_rate_best, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        min_child_weight = min_child_weight_best,
        #nthread=4,
        #tree_method='hist' 
        tree_method='gpu_hist' 
    )
    h = clf.fit(X_train1, y_train1, 
        eval_set=[(X_test1,y_test1)],
        verbose=50, early_stopping_rounds=100)

In [ ]:
if BUILD96:

    feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X_train.columns)), columns=['Value','Feature'])

    plt.figure(figsize=(20, 10))
    sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).iloc[:50])
    plt.title('XGB96 Most Important')
    plt.tight_layout()
    plt.show()
        
    

In [ ]:
feat = feature_imp.sort_values(by='Value', ascending=False)
feat_best = feat["Feature"][:350]
feat_best_list =feat_best.to_list()

In [ ]:
X_train_best = X_train[feat_best_list]
X_test_best = X_test[feat_best_list]
X_train_best.shape, X_test_best.shape

In [ ]:
del clf, h; x=gc.collect()

# MODEL AND PREDICTION

In [ ]:
groups  =X_train['DT_M']
group_kfold = GroupKFold(n_splits=6)
group_kfold.get_n_splits(X_train_best, y_train, groups)

print(group_kfold)

for train_index, test_index in group_kfold.split(X_train_best, y_train, groups):
    print("\nTRAIN:", train_index, "TEST:", test_index)
    X_train_df, X_test_df = X_train_best.iloc[train_index], X_train_best.iloc[test_index]
    y_train_df, y_test_df = y_train.iloc[train_index], y_train.iloc[test_index]
    
clf = xgb.XGBClassifier(
            n_estimators=n_estimators_best,
            max_depth=max_depth_best,
            learning_rate=learning_rate_best,
            subsample=0.8,
            colsample_bytree=0.4,
            missing=-1,
            eval_metric='auc',
            min_child_weight = min_child_weight_best,
            # USE CPU
            #nthread=4,
            #tree_method='hist'
            # USE GPU
            tree_method='gpu_hist' 
        )        
h = clf.fit(X_train_df, y_train_df, eval_set=[(X_test_df,y_test_df)],verbose=100, early_stopping_rounds=200)
    
oof = clf.predict_proba(X_train_df)[:,1]
preds = clf.predict_proba(X_test_best)[:,1]



In [ ]:
from sklearn import metrics


pred1 = clf.predict(X_test_df)
fpr, tpr, thresholds = metrics.roc_curve(y_test_df, pred1, pos_label=2)
metrics.auc(fpr, tpr)

print(metrics.confusion_matrix(y_test_df, pred1))
print(metrics.classification_report(y_test_df, pred1))

In [ ]:
plt.hist(oof,bins=100)
plt.ylim((0,5000))
plt.title('XGB OOF')
plt.show()

X_train['oof'] = oof
X_train.reset_index(inplace=True)
X_train[['TransactionID','oof']].to_csv('oof_xgb_96.csv')
X_train.set_index('TransactionID',drop=True,inplace=True)

In [ ]:
sample_submission = pd.read_csv('../input/ieee-fraud-detection/sample_submission.csv')
sample_submission.isFraud = preds
sample_submission.to_csv('sub_xgb_99.csv',index=False)

plt.hist(sample_submission.isFraud,bins=100)
plt.ylim((0,5000))
plt.title('XGB96 Submission')
plt.show()